#### Importing supporting Libraries 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import spacy

from matplotlib.pyplot import imread
from wordcloud import WordCloud,STOPWORDS
%matplotlib inline
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
p = PorterStemmer()
all_stopwords= stopwords.words('english')
all_stopwords.remove('not')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
#Read the file
review=pd.read_csv('chrome_reviews.csv')
review.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


### Preprocessing

In [3]:
# Droping the columns which are not necessery 
review=review.drop(['ID','Review URL','Thumbs Up','User Name','Developer Reply','Developer Reply','Review Date','App ID'],axis=1)
review

,Text,Star,Version
0,This is very helpfull aap.,5,83.0.4103.106
1,Good,3,85.0.4183.127
2,Not able to update. Neither able to uninstall.,1,85.0.4183.127
3,Nice app,4,77.0.3865.116
4,Many unwanted ads,1,87.0.4280.66
...,...,...,...
7199,Bagusss..,5,88.0.4324.93
7200,Bad version 😔,1,88.0.4324.181
7201,One thing that I have to say I can't spelled t...,5,89.0.4389.72
7202,Excellent,5,89.0.4389.72


In [4]:
review=review.drop(['Version'],axis=1)
review

,Text,Star
0,This is very helpfull aap.,5
1,Good,3
2,Not able to update. Neither able to uninstall.,1
3,Nice app,4
4,Many unwanted ads,1
...,...,...
7199,Bagusss..,5
7200,Bad version 😔,1
7201,One thing that I have to say I can't spelled t...,5
7202,Excellent,5


In [5]:
corpus=[]
n=len(review)
for i in range(0,n):
    sent = re.sub('[^a-zA-Z]', ' ',str(review['Text'][i]))
    sent = sent.lower()
    sent = sent.split()
    sent = [p.stem(word) for word in sent if not word in set(all_stopwords)]
    sent = ' '.join(sent)
    corpus.append(sent)

In [6]:
corpus

['helpful aap',
 'good',
 'not abl updat neither abl uninstal',
 'nice app',
 'mani unwant ad',
 'app good',
 'ye ye',
 'awesom',
 'bad app',
 'mani time tri updat not updat whenev tri use app freez goe back home',
 'good',
 'good app',
 'app not get updat not get open say u r use old version plz updat not updat',
 'mostli use',
 'everyth ia possibl chrome like',
 'come real statu thank',
 'good',
 'low download speed',
 'suuper',
 'want download greana free fire not com file',
 'good news etc',
 'best',
 'good',
 'mydeviceshavebeenhack im humbl christian know know hard need tell know two face hard take hear tri lie much world even onlin hide true tri get life without threat everyth know stori truth call car tel rick desire come see im done keep post r',
 'awesom browser',
 'rait crom',
 'ok',
 'continu awe snap fix',
 'move download use',
 'best fastest',
 'data consum',
 'phone samsung updat app',
 'lag day worst even updat even updat',
 'good app like everi use',
 'not bad',
 'nice 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 10000)

In [8]:
X = cv.fit_transform(corpus).toarray()
y = review.iloc[:, -1].values

In [9]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
y

array([5, 3, 1, ..., 5, 5, 1], dtype=int64)

In [30]:
# Saving dictionary to later use in prediction
import pickle
pickle.dump(cv, open('chrome_review_pridiction.pkl', "wb"))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [32]:
from sklearn.svm import LinearSVC
classifier = LinearSVC()
classifier.fit(X_train, y_train)

LinearSVC()

In [33]:
# Exporting NB Classifier to later use in prediction
import joblib
joblib.dump(classifier, 'Classifier_Sentiment_Model') 

['Classifier_Sentiment_Model']

In [34]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[ 397   10   17   10  132]
 [  42    2    4    3   39]
 [  48    5    2    2   89]
 [  24    1    0    5  146]
 [  62    3    9   23 1087]]


0.6905642923219242